In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
import pickle
import pandas as pd
from tasks import wsd
from pathlib import Path
from tasks import wsd
from utils import nlp_tools
from tqdm.auto import tqdm
import numpy as np
from flair.embeddings import TransformerWordEmbeddings
from utils.classificaton_utils import binarize, vectorize_target_expression,cosine_similiarity

In [43]:

tqdm.pandas()

lemma,pos = 'machine', "NN"
senses = {'machine_nn01-38475923'}
relations = ['seed','synonym'] # ,'descendant','sibling'

# whether we use only information on the lemma for the predictive model (e.g. only the lemma senses definitions for lesk baselines)
eval_mode = "lemma_etal" # lemma or lemma_etal

df_train, df_val, df_test = binarize(lemma,
                        pos,
                        senses, 
                        relations,
                        strict_filter=True,
                        start=1700,
                        end=1920,
                        eval_mode=eval_mode)

Index(['sense_id', 'lemma_definition', 'definition', 'word_id', 'lemma',
       'quotation_id', 'source', 'text', 'year'],
      dtype='object')
# senses before filtering by date = 517
# senses after filtering by date = 368


# of seed senses 23 
# of synonyms 324 
# of branch senses 0


# of seeds selected 1 
# of synonyms selected 8 
# of branches selected 0


In [44]:
df_train.head()

,sense_id,lemma_definition,definition,word_id,lemma,quotation_id,source,text,year,label,id,daterange,provenance,provenance_type,relation_to_core_senses,relation_to_seed_senses,full_text,keyword,keyword_offset
0,power_nn01-28687561,Ability to act or affect something strongly; p...,Mathematics. A quantity obtained by multiplyin...,power_nn01,power,power_nn01-28687678,"{'title': 'London, Edinb., & Dublin Philos. Ma...","{'keyword': 'power', 'full_text': 'The scatter...",1913.0,0,NaN,NaN,NaN,NaN,NaN,NaN,The scattering coefficient varied inversely as...,power,58
1,machine_nn01-38474140,"A complex device, consisting of a number of in...","A living body, esp. the human body considered ...",machine_nn01,machine,machine_nn01-38474186,"{'title': 'Lexicon Physico-medicum', 'author':...","{'keyword': 'Machine', 'full_text': 'Until som...",1722.0,0,machine_nn01-38474140,"{'end': None, 'start': 1604, 'obsolete': False...","[[machine_nn01-38474140, seed, machine_nn01]]",seed,{machine_nn01-38474140},{machine_nn01-38474140},Until some Authors..have demonstrated the Laws...,Machine,75
2,power_nn01-28686646,Ability to act or affect something strongly; p...,A powerful celestial or spiritual being; a god...,power_nn01,power,power_nn01-28686756,"{'title': 'Odyssey', 'author': 'A. Pope', 'gen...","{'keyword': 'pow'rs', 'full_text': 'There land...",1725.0,0,NaN,NaN,NaN,NaN,NaN,NaN,"There land, and pay due victims to the pow'rs.",pow'rs,39
3,machine_nn01-38476316,"A complex device, consisting of a number of in...",slang. The penis; the female genitals (rare). ...,machine_nn01,machine,machine_nn01-38476322,"{'title': 'Mem. Woman of Pleasure', 'author': ...","{'keyword': 'machine', 'full_text': 'Coming ou...",1749.0,0,machine_nn01-38476316,"{'end': None, 'start': 1749, 'obsolete': False...","[[machine_nn01-38476316, seed, machine_nn01]]",seed,{machine_nn01-38476316},{machine_nn01-38476316},Coming out with that formidable machine of his...,machine,32
4,power_nn01-28688218,Ability to act or affect something strongly; p...,Capacity for exerting mechanical force or doin...,power_nn01,power,power_nn01-28688241,"{'title': 'Let.', 'author': 'M. Edgeworth', 'g...","{'keyword': 'power', 'full_text': 'Sydney Smit...",1822.0,0,NaN,NaN,NaN,NaN,NaN,NaN,Sydney Smith to frighten a little boy who was ...,power,181


In [45]:
model_name = 'bert-base-uncased'
layers="-1"
pooling_operation='mean'



embedding_type = TransformerWordEmbeddings(model_name,
                                            layers=layers,
                                            pooling_operation=pooling_operation)

In [46]:
df_train = vectorize_target_expression(df_train,embedding_type)
df_val = vectorize_target_expression(df_val,embedding_type)
df_test = vectorize_target_expression(df_test,embedding_type)

[INFO] swifter is installed. Parallelize pandas apply method.
[WARNING] could not find swifter...run pandas apply on one process.
[INFO] swifter is installed. Parallelize pandas apply method.
[WARNING] could not find swifter...run pandas apply on one process.
[INFO] swifter is installed. Parallelize pandas apply method.
[WARNING] could not find swifter...run pandas apply on one process.


In [50]:
df_test.vector

108    [-0.23773926, 0.76330495, -0.18839866, -0.1579...
316    [0.108669445, -0.036040783, 0.042272598, 0.319...
247    [0.68850386, 0.3318206, 0.30432534, 0.16182785...
97     [0.16162382, 0.118292086, 0.044132993, 0.17878...
57     [0.83684593, 0.5584122, -0.16416408, 0.2444155...
                             ...                        
258    [0.22711894, -0.17166136, 0.010308429, 0.51724...
115    [0.047639232, 0.7235628, 0.004915458, 0.109136...
188    [0.5243632, -0.0057972977, 0.11866429, 0.86185...
18     [0.08796091, 0.8054311, 0.18142846, 0.7897241,...
328    [0.5455518, 0.0035560497, 0.012848668, 0.15377...
Name: vector, Length: 66, dtype: object

In [56]:
def bert_polar_vector(vector,polar_vectors):
    return str(np.argmax(quotation_df_avg_by_label.apply(cosine_similiarity, target = vector)))

polar_vectors = df_train.groupby('label')['vector'].apply(np.mean,axis=0)
polar_vectors


label
0    [0.20609632, -0.07717068, 0.07131706, 0.166867...
1    [0.057824414, -0.20342115, 0.09946487, 0.08246...
Name: vector, dtype: object

In [57]:
df_test["bert_polar"] = df_test.vector.progress_apply(bert_polar_vector, polar_vectors = polar_vectors)
df_test["bert_polar"]

108    0
316    1
247    1
97     0
57     0
      ..
258    1
115    0
188    0
18     0
328    0
Name: bert_polar, Length: 66, dtype: object

In [58]:
wsd.eval("bert_polar",df_test)

{'1': [0.353, 0.857, 0.5], '0': [0.98, 0.814, 0.889]}